# Azure Machine Learning Jobs Explorer

## Before you start

You'll need the latest version of the **azure-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [ ]:
pip install azure-ai-ml azure-identity pandas mlflow azureml-mlflow

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.26.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/jupyter_env/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()


In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## List Jobs Runs

In [4]:
runs = [run for run in ml_client.jobs.list()]
len(runs)

69

## Print job data

In [5]:
print(runs[0])

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


type: command
outputs:
  default:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.dynamic_apple_3kpvlg1l83
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
resources:
  instance_count: 1
  shm_size: 2g
component:
  name: dynamic_apple_3kpvlg1l83
  display_name: diabetes-pythonv2-train
  tags:
    envType: test-env-1
    _aml_system_ComputeTargetStatus: '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":2}'
  type: command
  outputs:
    default:
      type: uri_folder
      mode: rw_mount
  command: python diabetes-training.py
  environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
  code: azureml:/subscriptions/a647c11c-fe4c-43c4-b672-963b71adab36/resourceGroups/my-machine-learning-rg-eastus2-112024/providers/Microsoft.MachineLearningServices/workspaces/my-azure-ml-ws-eastus2-112024/codes/17926021-1345-41b9-919e-cb1b7af5f700/versions/1
  creati

In [6]:
from datetime import datetime

def calc_job_duration(job_start_date, job_end_date):
    start_date = datetime.strptime(job_start_date, "%Y-%m-%d %H:%M:%S")
    end_date = datetime.strptime(job_end_date, "%Y-%m-%d %H:%M:%S")
    difference = end_date - start_date
    return(difference.seconds)

In [7]:
for run in runs[0:1]:
    print(f"name: {run.component.name}")
    print(f"type: {run.type}")
    print(f"display_name: {run.component.display_name}")
    print(f"envType: {run.component.tags}")
    print(f"compute: {run.compute}")
    print(f"StartTimeUtc: {run.properties['StartTimeUtc']}")
    print(f"EndTimeUtc: {run.properties['EndTimeUtc']}")
    print(f"Job Duration (Seconds): {calc_job_duration(run.properties['StartTimeUtc'], run.properties['EndTimeUtc'])}")

name: dynamic_apple_3kpvlg1l83
type: command
display_name: diabetes-pythonv2-train
envType: {'envType': 'test-env-1', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":2}'}
compute: aml-cluster
StartTimeUtc: 2025-03-23 12:13:04
EndTimeUtc: 2025-03-23 12:13:12
Job Duration (Seconds): 8


## Prepare a list of jobs

In [8]:
runs_list = []
for run in runs:
    if 'command' in run.type:
        job = dict(name=run.component.name, 
                 run_type=run.type,  
                 display_name=run.component.display_name, 
                 environment=run.environment,
                 tag=run.component.tags,
                 compute=run.compute,
                 duration_sec=calc_job_duration(run.properties['StartTimeUtc'], run.properties['EndTimeUtc'])
        )
        
        runs_list.append(job)

In [9]:
len(runs_list)

61

In [10]:
runs_list[0]

{'name': 'dynamic_apple_3kpvlg1l83',
 'run_type': 'command',
 'display_name': 'diabetes-pythonv2-train',
 'environment': 'azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest',
 'tag': {'envType': 'test-env-1',
  '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":2}'},
 'compute': 'aml-cluster',
 'duration_sec': 8}

## Create a dataframe with jobs data

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(runs_list)

In [13]:
df

,name,run_type,display_name,environment,tag,compute,duration_sec
0,dynamic_apple_3kpvlg1l83,command,diabetes-pythonv2-train,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'envType': 'test-env-1', '_aml_system_Compute...",aml-cluster,8
1,olden_fish_4q2vxwl4bh,command,diabetes-pythonv2-train,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'envType': 'test-env-1', '_aml_system_Compute...",aml-cluster,16
2,busy_arm_ml2zxk4rzb,command,diabetes-pythonv2-train,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'envType': 'test-env-1', '_aml_system_Compute...",aml-cluster,9
3,affable_window_nlp1l4vt7h,command,diabetes-pythonv2-train,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'envType': 'test-env-1', '_aml_system_Compute...",aml-cluster,9
4,tidy_vinegar_6x5hwk37jr,command,diabetes-pythonv2-train,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'envType': 'test-env-1', '_aml_system_Compute...",aml-cluster,8
...,...,...,...,...,...,...,...
56,magenta_eye_67wq3qycmg,command,diabetes-train-mlflow,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'model_type': 'LogisticRegression', '_aml_sys...",aml-cluster,13
57,patient_basin_jdnl1p9w2h,command,diabetes-train-script,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'_aml_system_ComputeTargetStatus': '{""Allocat...",aml-cluster,72
58,purple_eye_tdznk0q9m9,command,move-diabetes-data,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'_aml_system_ComputeTargetStatus': '{""Allocat...",aml-cluster,131
59,icy_arch_5s2wwdzw1d,command,move-diabetes-data,azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-...,"{'_aml_system_ComputeTargetStatus': '{""Allocat...",aml-cluster,123


## Get experiments data

In [14]:
import mlflow
experiments = mlflow.search_experiments()
for exp in experiments:
    print(exp.name)

/anaconda/envs/jupyter_env/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


diabetes-training
move-diabetes-data
mlflow-experiment-diabetes
sweep-diabetes
pipeline_diabetes
iris
dpv2-classifier-experiment
auto-ml-class-dev
dpv2-regression-experiment
automobile-price-prediction
pipeline_samples
prepare_image
YOLOX-experiment
explore-images-data
explore-images-data-compute-instance


In [15]:
exp = mlflow.get_experiment_by_name(experiments[0].name)
print(exp)
print(exp.experiment_id)

<Experiment: artifact_location='', creation_time=1731841973255, experiment_id='e99e4d39-4611-4b0a-b78c-08b0edb8c3ec', last_update_time=None, lifecycle_stage='active', name='diabetes-training', tags={}>
e99e4d39-4611-4b0a-b78c-08b0edb8c3ec


In [16]:
mlflow.search_runs(exp.experiment_id).head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.Accuracy,metrics.AUC,metrics.training_recall_score,metrics.training_roc_auc_score,...,params.class_weight,params.warm_start,tags.mlflow.user,tags.mlflow.rootRunId,tags.mlflow.runName,tags.model_type,tags.mlflow.autologging,tags.estimator_class,tags.estimator_name,tags.envType
0,careful_calypso_kfxyvw94vx,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 11:16:47.225000+00:00,2024-11-17 11:18:03.437000+00:00,NaN,NaN,NaN,NaN,...,None,None,System Administrator,careful_calypso_kfxyvw94vx,diabetes-pythonv2-train,None,None,None,None,None
1,patient_basin_jdnl1p9w2h,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:51:12.086000+00:00,2024-11-17 12:52:24.965000+00:00,NaN,NaN,NaN,NaN,...,None,None,System Administrator,patient_basin_jdnl1p9w2h,diabetes-train-script,None,None,None,None,None
2,magenta_eye_67wq3qycmg,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:52:38.401000+00:00,2024-11-17 12:52:51.092000+00:00,0.774,0.84832,NaN,NaN,...,None,None,System Administrator,magenta_eye_67wq3qycmg,diabetes-train-mlflow,LogisticRegression,None,None,None,None
3,clever_tangelo_xs41b96z3z,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:53:01.277000+00:00,2024-11-17 12:54:20.424000+00:00,NaN,NaN,0.790857,0.861979,...,None,False,System Administrator,clever_tangelo_xs41b96z3z,diabetes-train-autolog,None,sklearn,sklearn.linear_model._logistic.LogisticRegression,LogisticRegression,None
4,coral_peach_6j5tl7czv7,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:58:26.887000+00:00,2024-11-17 12:59:41.709000+00:00,NaN,0.84832,NaN,NaN,...,None,None,System Administrator,coral_peach_6j5tl7czv7,diabetes-train-mlflow,LogisticRegression,None,None,None,None


## Create a dataframe with all experiments and experiment name

In [17]:
first_time = True
experiments = mlflow.search_experiments()
for exp in experiments:
    runs = mlflow.search_runs(exp.experiment_id)
    runs['experiment_name'] = exp.name
    print(exp.name)
    if first_time:
        all_runs_df = runs
        first_time = False
    else:
        all_runs_df = pd.concat([all_runs_df, runs], axis=0, ignore_index=True)

diabetes-training
move-diabetes-data
mlflow-experiment-diabetes
sweep-diabetes
pipeline_diabetes
iris
dpv2-classifier-experiment
auto-ml-class-dev
dpv2-regression-experiment
automobile-price-prediction
pipeline_samples
prepare_image
YOLOX-experiment
explore-images-data
explore-images-data-compute-instance


In [18]:
all_runs_df.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.Accuracy,metrics.AUC,metrics.training_recall_score,metrics.training_roc_auc_score,...,metrics.mean_absolute_error,tags._azureml.ComputeTargetType,metrics.Coefficient_of_Determination,metrics.Mean_Absolute_Error,metrics.Relative_Absolute_Error,metrics.Root_Mean_Squared_Error,metrics.Relative_Squared_Error,params.training_learning_rate,params.learning_rate_schedule,params.training_max_epochs
0,careful_calypso_kfxyvw94vx,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 11:16:47.225000+00:00,2024-11-17 11:18:03.437000+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,patient_basin_jdnl1p9w2h,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:51:12.086000+00:00,2024-11-17 12:52:24.965000+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,magenta_eye_67wq3qycmg,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:52:38.401000+00:00,2024-11-17 12:52:51.092000+00:00,0.774,0.84832,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,clever_tangelo_xs41b96z3z,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:53:01.277000+00:00,2024-11-17 12:54:20.424000+00:00,NaN,NaN,0.790857,0.861979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,coral_peach_6j5tl7czv7,e99e4d39-4611-4b0a-b78c-08b0edb8c3ec,FINISHED,,2024-11-17 12:58:26.887000+00:00,2024-11-17 12:59:41.709000+00:00,NaN,0.84832,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
all_runs_df[["run_id","experiment_name"]]

,run_id,experiment_name
0,careful_calypso_kfxyvw94vx,diabetes-training
1,patient_basin_jdnl1p9w2h,diabetes-training
2,magenta_eye_67wq3qycmg,diabetes-training
3,clever_tangelo_xs41b96z3z,diabetes-training
4,coral_peach_6j5tl7czv7,diabetes-training
...,...,...
128,loving_nutmeg_5lyy7j8gqr,explore-images-data
129,sweet_okra_23ntwkhggy,explore-images-data
130,shy_caravan_94mrr1tn1r,explore-images-data
131,hungry_rocket_vfx2ng9ljk,explore-images-data
